In [27]:
# 원티드 채용공고 크롤링
import requests
import datetime
import json

def crawling_wanted_data(parsed_data, info_keys):
    result={}
    for key in info_keys:
        if key == 'company':   # 회사 이름
            result['company'] = parsed_data['job']['company']['name']  
        elif key == 'intro':   # 회사 소개
            result['intro'] = parsed_data['job']['detail']['intro'] 
        elif key == 'location':  # 근무 지역
            result['location'] = parsed_data['job']['address']['full_location'] 
        elif key == 'title':  # 제목
            result['title'] = parsed_data['job']['position']
        elif key == 'main_job': # 주요업무
            result['main_job'] = parsed_data['job']['detail']['main_tasks']
        elif key == 'require':  # 자격요건
            result['require'] = parsed_data['job']['detail']['requirements']
        elif key == 'prefer_point':  # 우대사항
            result['prefer_point'] = parsed_data['job']['detail']['preferred_points']
        elif key == 'benefits':  # 혜택 및 복지
            result['benefits'] = parsed_data['job']['detail']['benefits']
        elif key == 'deadline':  # 마감날짜 None = 상시
            result['deadline'] = parsed_data['job']['due_time']
        elif key == 'geo_location': # 근무지역 (위도,경도)
            geo_location = parsed_data['job']['address'].get('geo_location')
            if geo_location:
                result['lat'] = geo_location['n_location']['lat']
                result['lng'] = geo_location['n_location']['lng']
            else:
                result['lat'] = None
                result['lng'] = None
        elif key == 'url':
            base = "https://www.wanted.co.kr/wd/"
            result['url'] = f"{base}{parsed_data['job']['id']}"
    return result

parsed_data = []

timestamp = int(datetime.datetime.now().timestamp())
tag_type_ids = 518
years = -1
limit = 10
offset = 0
# tag_type_ids: 518 개발/ 507 경영-비즈니스 /523 마케팅 /511 디자인 /530 영업 /510 고객서비스-리테일 /524 미디어 /513 엔지니어링-설계
# 517 HR /959 게임 제작 /508 금융 /522 제조-생산 /515 의료-제약-바이오 /10101 교육 /532 물류-무역 /10057 식-음료 /521 법률-법집행기관 /509 건설-시설 /514 공공-복지
# limit -> 100이면 100개씩 데이터추출 (한번에 100밖에 불가)
# offset -> 0번째부터 추출

# 10000개 데이터 가져오기 
total_data_count = 10

base_url = "https://www.wanted.co.kr/api/v4/jobs/"
for offset in range(0, total_data_count, limit):
    
    url = "https://www.wanted.co.kr/api/v4/jobs?{}&country=kr&tag_type_ids={}&job_sort=job.latest_order&locations=all&years={}&limit={}&offset={}".format(timestamp, tag_type_ids, years, limit, offset)
    parse_company = requests.get(url).json()
    
    recur_ids = [temp['id'] for temp in parse_company['data']]
    

    for recur_id in recur_ids:
        detail_url = f"{base_url}{recur_id}?{timestamp}"
        job_data = requests.get(detail_url).json()

        info_keys = ['company', 'title','intro', 'location', 'main_job','require', 'prefer_points', 'benefits','deadline', 'geo_location','url']  # 필요한 정보 키
        crawled_data = crawling_wanted_data(job_data, info_keys)
        
        print(json.dumps(crawled_data, indent=4, ensure_ascii=False))
        


{
    "company": "오픈플랜",
    "title": "Back-End Junior 개발자",
    "intro": "토글서비스는,\n레저, 여행, 운전, 반려동물 등 일상생활에 필요한 다양한 미니보험 상품을 쉽게 사용할 수 있는 보험 플랫폼입니다.\n\n기존의 보험 가입 방식은 보험에 대해 이해하기 어려웠을 뿐만 아니라, 장기간의 납입에 대한 부담이 있어 사용자에게 큰 진입장벽을 느끼게 하였습니다. 이에 저희는 실생활에 필요한 모든 보험을 한 곳에서 검색하고 쉽고, 빠르고, 원하는 만큼만 선택하여 가입할 수 있는 생활밀착형 보험 서비스를 제공하고 있습니다.\n\n\n오픈플랜의 현재\n저희 오픈플랜은 2019년 11월에 설립된 회사로, 3년동안 수 차례의 투자유치를 성공적으로 진행하였으며, 최근에는 중소벤처기업부 주최의 기술창업지원 프로그램 TIPS에 선정되었습니다.\n현재는 시리즈B 투자유치를 준비중에 있으며, 이에 발 맞추어 다음과 같은 분들의 적극적인 지원을 기다리고 있습니다.​",
    "location": "서울 성동구 아차산로11길 18, 907호",
    "main_job": "• Service 내의 Business Flow 구현\n• Service 접근에 필요한 API 구현\n• API 구현을 위한 Data Model 설계\n• Service 운영을 위한 API 구현\n• Service 운영을 위한 Back Office 구현\n\nJava 또는 Kotlin, Spring boot(Framework), JPA(query/dsl, etc), RESTful etc.",
    "require": "• 학력 : 대졸 이상\n• CS에 대한 이해",
    "benefits": "•  중식 제공\n•  매월 마지막주 금요일은 12시 30분에 퇴근 합니다.\n•  자유로운 복장 및 연차 (반반차 사용가능)\n•  도서 구매 자격증 교육 시험비 지원",
    "deadline": null,
    "lat": 37.5452509,
   